# World Data League 2023

## Notebook Submission Template

This notebook is one of the mandatory deliverables when you submit your solution. Its structure follows the WDL evaluation criteria and it has dedicated cells where you should add information. Make sure your code is readable as it will be the only technical support the jury will have to evaluate your work. Make sure to list all the datasets used besides the ones provided.

Instructions:
1. 🧱 Create a separate copy of this template and **do not change** the predefined structure
2. 👥 Fill in the Authors section with the name of each team member
3. 💻 Develop your code - make sure to add comments and save all the output you want the jury to see. Your code **must be** runnable!
4. 📄 Fill in all the text sections
5. 🗑️ Remove this section (‘Notebook Submission Template’) and any instructions inside other sections
6. 📥 Export as HTML and make sure all the visualisations are visible.
7. ⬆️ Upload the .ipynb file to the submission platform and make sure that all the visualisations are visible and everything (text, images, ..) in all deliverables renders correctly.


## 🎯 Challenge
Determining The Main Mobility Flows in the City of Lisbon Based on Mobile Device Data

## Team: (Insert Team Name Here)
## 👥 Authors
* Eduardo Silva
* Pedro Lopes
* Tomás Pereira

## 💻 Development
Start coding here! 🐱‍🏍

Create the necessary subsections (e.g. EDA, different experiments, etc..) and markdown cells to include descriptions of your work where you see fit. Comment your code. 

All new subsections must start with three hash characters. More specifically, don't forget to explore the following:
1. Assess the data quality
2. Make sure you have a good EDA where you enlist all the insights
3. Explain the process for feature engineering and cleaning
4. Discuss the model / technique(s) selection
5. Don't forget to explore model interpretability and fairness or justify why it is not needed

Pro-tip 1: Don't forget to make the jury's life easier. Remove any unnecessary prints before submitting the work. Hide any long output cells (from training a model for example). For each subsection, have a quick introduction (justifying what you are about to do) and conclusion (results you got from what you did). 

Pro-tip 2: Have many similiar graphs which all tell the same story? Add them to the appendix and show only a couple of examples, with the mention that all the others are in the appendix.

Pro-tip 3: Don't forget to have a motivate all of your choices, these can be: Data-driven, constraints-driven, literature-driven or a combination of any. For example, why did you choose to test certain algorithms or why only one.

In [ ]:
import json
import pandas as pd
import geopandas as gpd
import typing
from geopy.distance import geodesic as GD 
from shapely.geometry import Point, Polygon
from sklearn.metrics import DistanceMetric
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import math
import folium
import geopandas as gpd 
from shapely.geometry import Polygon
import pyproj
import math
import lightgbm as lgb
from skopt import gp_minimize
from skopt.space import Real, Integer
import os
from dotenv import load_dotenv
load_dotenv() 

CRS = "EPSG:4326"

In [ ]:
# PATHS
mobile_devices_sep_path = os.environ["CML_DISPOSITIVOS MOVEIS_GRELHA_indCD15m_2022_09_0001_4000"] # DATASET 7 September
mobile_devices_oct_path = os.environ["CML_DISPOSITIVOS MOVEIS_GRELHA_indCD15m_2022_10_0001_4000"] # DATASET 7 OCTOBER
mobile_devices_nov_path = os.environ["CML_DISPOSITIVOS MOVEIS_GRELHA_indCD15m_2022_11_0001_4000"] # DATASET 7 November

mobile_devices_proc_sep_path = os.environ["RESULTS_09"]
mobile_devices_proc_oct_path = os.environ["RESULTS_10"]
mobile_devices_proc_nov_path = os.environ["RESULTS_11"]

metro_station_path = os.environ["METRO_STATATION"] # EXTERNAL DATASET

grid_path = os.environ["DISPOSITIVOS MOVEIS_QUADRICULAS"]

### EDA

In [ ]:
mobile_devices_sept = pd.read_csv(mobile_devices_sep_path, nrows=100000) # MODIFY TO USE ALL THE DATA HERE
mobile_devices_oct = pd.read_csv(mobile_devices_sep_path, nrows=100) # MODIFY TO USE ALL THE DATA HERE
mobile_devices_nov = pd.read_csv(mobile_devices_sep_path, nrows=100) # MODIFY TO USE ALL THE DATA HERE

mobile_devices = pd.concat([mobile_devices_sept, mobile_devices_oct, mobile_devices_nov])

In [ ]:
def prepare_mobile_device_dataset(mobile_devices):
    # Format columns
    conversion_columns = {
        "C1": "n_terminals", 
        "C2": "n_terminals_roaming",
        "C3": "terminals_remain",
        "C4": "terminals_remain_roaming",
        "C5": "in_flow", 
        "C6": "out_flow",
        "C7": "in_flow_roaming", 
        "C8": "out_flow_roaming"
        }
    mobile_devices = mobile_devices.rename(columns=conversion_columns)
    mobile_devices["total_flow"] = mobile_devices["in_flow"] + mobile_devices["out_flow"]

    mobile_devices["Datetime"] = pd.to_datetime(mobile_devices["Datetime"], format="%Y-%m-%dT%H:%M:%S.%fZ")
    mobile_devices["weekday"] = mobile_devices["Datetime"].dt.dayofweek
    mobile_devices["time_of_day"] = mobile_devices["Datetime"].dt.hour

    return mobile_devices

In [ ]:
mobile_devices = prepare_mobile_device_dataset(mobile_devices)
rush_hours_df = mobile_devices # UNCOMMENT FOLLOWING LINE TO ADD RUSH HOUR FILTER
# rush_hours_df = mobile_devices[(mobile_devices["weekday"].isin([0,1,2,3,4]) & mobile_devices["time_of_day"].isin([7,8,9,10,17,18,19,20]))]

In [ ]:
print(len(mobile_devices))
print(len(rush_hours_df))

In [ ]:
columns = ['n_terminals', 'n_terminals_roaming', 'terminals_remain', 'terminals_remain_roaming', 'in_flow', 'out_flow', 'in_flow_roaming', 'out_flow_roaming', 'total_flow']
pivot = pd.pivot_table(
    data=rush_hours_df,
    index='time_of_day',
    aggfunc='sum',
    values=columns
)

for col in columns:
    pivot[col].plot(kind='bar', width=1.0)
    plt.savefig(f'{col}.png')
    plt.clf()

### DATA PREPARATION

In [ ]:
# ALREADY AT INTERMEDIATE STEP DATA LOADING
grid_df = pd.read_excel(grid_path)

# INTERMEDIATE STEPS DATASET
# mobile_devices_sept = pd.read_csv(mobile_devices_proc_sep_path)
# mobile_devices_oct = pd.read_csv(mobile_devices_proc_oct_path)
# mobile_devices_nov = pd.read_csv(mobile_devices_proc_nov_path)
# mobile_devices = pd.concat([mobile_devices_sept, mobile_devices_oct, mobile_devices_nov])
# mobile_devices = mobile_devices[(mobile_devices["weekday"].isin([0,1,2,3,4]) & mobile_devices["time_of_day"].isin([7,8,9,10,17,18,19,20]))]


#### GRID AND TERMINALS DATASETS

In [ ]:
def create_grid_id_matrix():
    # Create a list of tuples
    tuples = grid_df.apply(lambda row: (row["grelha_id"], row["latitude"], row["longitude"]), axis=1).tolist()
    # Sort the tuples by latitude and longitude
    tuples.sort(key=lambda x: (x[1], x[2]))

    latitude_tuples = {ele[1] for ele in tuples}
    longitude_tuples = {ele[2] for ele in tuples}

    # Define the boundaries of the matrix
    lat_min, lat_max = max(grid_df["latitude"]), min(grid_df["latitude"])
    lon_min, lon_max = max(grid_df["longitude"]), min(grid_df["longitude"])

    # Calculate the size of each cell in latitude and longitude degrees
    lat_size = (lat_max - lat_min) / len(latitude_tuples)
    lon_size = (lon_max - lon_min) / len(longitude_tuples)

    # Create a 3x3 matrix with NaN values
    matrix = np.empty((len(latitude_tuples), len(longitude_tuples)))
    matrix[:] = np.nan

    # Populate the matrix with the sorted tuples
    for i, t in enumerate(tuples):
        grid_id, lat, lon = t
        lat_idx = int((lat - lat_min) // lat_size) - 1
        lon_idx = int((lon - lon_min) // lon_size) - 1
        matrix[lat_idx, lon_idx] = i

    grid_id_matrix = []
    for arr in matrix:
        grid_id_line = []

        for i in arr:
            if math.isnan(i):
                grid_id_line.append(np.nan)
            else:
                grid_id_line.append(tuples[int(i)][0])

        grid_id_matrix.append(grid_id_line)

    return grid_id_matrix

def get_adjacent_grid_ids(matrix, d1_pos, d2_pos):
    # d1 is the first dimension, so vertical axis
    # d1 is the second dimension, so horizontal axis
    max_d1 = len(matrix) - 1
    max_d2 = len(matrix[0]) - 1

    assert d1_pos >= 0
    assert d1_pos <= max_d1
    assert d2_pos >= 0
    assert d2_pos <= max_d2

    adjacent_grid_id_list = []

    up_d1_pos = d1_pos - 1 if (d1_pos - 1) >= 0 else -1
    up_d2_pos = d2_pos

    if up_d1_pos != -1 and not math.isnan(matrix[up_d1_pos][up_d2_pos]):
        adjacent_grid_id_list.append(matrix[up_d1_pos][up_d2_pos])

    down_d1_pos = d1_pos + 1 if (d1_pos + 1) <= max_d1 else -1
    down_d2_pos = d2_pos

    if down_d1_pos != -1 and not math.isnan(matrix[down_d1_pos][down_d2_pos]):
        adjacent_grid_id_list.append(matrix[down_d1_pos][down_d2_pos])

    left_d1_pos = d1_pos
    left_d2_pos = d2_pos - 1 if (d2_pos - 1) >= 0 else -1

    if left_d2_pos != -1 and not math.isnan(matrix[left_d1_pos][left_d2_pos]):
        adjacent_grid_id_list.append(matrix[left_d1_pos][left_d2_pos])

    right_d1_pos = d1_pos
    right_d2_pos = d2_pos + 1 if (d2_pos + 1) <= max_d2 else -1

    if right_d1_pos != -1 and not math.isnan(matrix[right_d1_pos][right_d2_pos]):
        adjacent_grid_id_list.append(matrix[right_d1_pos][right_d2_pos])

    up_left_d1_pos = d1_pos - 1 if (d1_pos - 1) >= 0 else -1
    up_left_d2_pos = d2_pos - 1 if (d2_pos - 1) >= 0 else -1

    if up_left_d1_pos != -1 and up_left_d1_pos != -1 and not math.isnan(matrix[up_left_d1_pos][up_left_d2_pos]):
        adjacent_grid_id_list.append(matrix[up_left_d1_pos][up_left_d2_pos])

    up_right_d1_pos = d1_pos - 1 if (d1_pos - 1) >= 0 else -1
    up_right_d2_pos = d2_pos + 1 if (d2_pos + 1) <= max_d2 else -1

    if up_right_d1_pos != -1 and up_right_d2_pos != -1 and not math.isnan(matrix[up_right_d1_pos][up_right_d2_pos]):
        adjacent_grid_id_list.append(matrix[up_right_d1_pos][up_right_d2_pos])

    down_left_d1_pos = d1_pos + 1 if (d1_pos + 1) <= max_d1 else -1
    down_left_d2_pos = d2_pos - 1 if (d2_pos - 1) >= 0 else -1

    if down_left_d1_pos != -1 and down_left_d2_pos != -1 and not math.isnan(matrix[down_left_d1_pos][down_left_d2_pos]):
        adjacent_grid_id_list.append(matrix[down_left_d1_pos][down_left_d2_pos])

    down_right_d1_pos = d1_pos + 1 if (d1_pos + 1) <= max_d1 else -1
    down_right_d2_pos = d2_pos + 1 if (d2_pos + 1) <= max_d2 else -1

    if (
        down_right_d1_pos != -1
        and down_right_d2_pos != -1
        and not math.isnan(matrix[down_right_d1_pos][down_right_d2_pos])
    ):
        adjacent_grid_id_list.append(matrix[down_right_d1_pos][down_right_d2_pos])

    return adjacent_grid_id_list


def get_adjacent_grid_dict(grid_id_matrix):
    adjacent_grid_dict = {}
    for i in range(len(grid_id_matrix)):
        for j in range(len(grid_id_matrix[0])):
            if not math.isnan(grid_id_matrix[i][j]):
                adjacent_grid_dict[grid_id_matrix[i][j]] = get_adjacent_grid_ids(grid_id_matrix, i, j)

    return adjacent_grid_dict

In [ ]:
def get_adjacent_data(group):
    current_id = group["Grid_ID"].iloc[0]

    adjacent_square_list = adjacent_grid_dict[current_id]

    adjacent_square_df = mobile_devices[mobile_devices["Grid_ID"].isin(adjacent_square_list)]

    data_columns = [
        "n_terminals",
        "n_terminals_roaming",
        "terminals_remain",
        "terminals_remain_roaming",
        "in_flow",
        "out_flow",
        "in_flow_roaming",
        "out_flow_roaming",
        "total_flow",
    ]
    data_groups = adjacent_square_df.groupby("Datetime")

    aggregation = data_groups[data_columns].aggregate("sum")
    aggregation["Grid_ID"] = current_id

    merged_df = pd.merge(group, aggregation, on=["Grid_ID", "Datetime"], suffixes=["", "_adjacent"])

    return merged_df

In [ ]:
# Create auxiliar info
grid_id_matrix = create_grid_id_matrix()
adjacent_grid_dict = get_adjacent_grid_dict(grid_id_matrix)

# Get Aggregated data
grouped_df = mobile_devices.groupby("Grid_ID")

result_df = grouped_df.apply(get_adjacent_data)

result_df.head()

#### Metro DATASET MERGE

In [ ]:
def convert_square_center_to_polygon(point: Point, square_length: int) -> Polygon:
    """Converts a grid square's center Point to a spacial Polygon.

    Args:
        point (Point): A square's center point (with longitude / latitude information).
        square_length (int): Square's side length.

    Returns:
        Polygon: The polygon representing the grid square.
    """

    half_length = square_length / 2

    # Get center-aligned top and bottom points
    center_north = GD(meters=half_length).destination((point.x, point.y), bearing=0)
    center_south = GD(meters=half_length).destination((point.x, point.y), bearing=180)

    # Get top corners
    north_east = GD(meters=half_length).destination(center_north, 90)
    north_west = GD(meters=half_length).destination(center_north, 270)

    # Get bottom corners
    south_east = GD(meters=half_length).destination(center_south, 90)
    south_west = GD(meters=half_length).destination(center_south, 270)


    return Polygon([north_east, south_east, south_west, north_west])

In [ ]:
def parse_squares_dataset(file_path: str) -> gpd.GeoDataFrame:
    """Parses the Lisbon grid squares dataset.

    Args:
        file_path (str): The path to file.

    Returns:
        gpd.GeoDataFrame: A GeoDataFrame containing the information of each grid square represented by spacial polygons.
    """
    # read the CSV file with the square information
    # squares_df = pd.read_csv(file_path)
    squares_df = pd.read_excel(file_path)

    # create a Point object for each square"s location
    squares_df["center_point"] = [Point(xy) for xy in zip(squares_df["longitude"], squares_df["latitude"])]
    squares_gdf = gpd.GeoDataFrame(squares_df, crs=CRS, geometry="center_point")

    squares_gdf["geometry"] = squares_gdf["center_point"].apply(lambda x: convert_square_center_to_polygon(x, 200))
    squares_gdf.set_geometry("geometry", inplace=True)
    squares_gdf.drop("center_point", axis=1, inplace=True)

    return squares_gdf

In [ ]:
def parse_metro_stations_file(file_path: str) -> gpd.GeoDataFrame:
    """Parses JSON file containing metro stations information in the city of Lisbon.

    Args:
        file_path (str): The path to the file.

    Returns:
        gpd.GeoDataFrame: A GeoDataFrame containing the information regarding the metron stations.
    """
    with open(file_path) as f:
        data = json.load(f)

    features = data["features"]
    rows = []

    for feature in features:
        properties = feature["properties"]
        nome = properties["NOME"]
        coordinates = feature["geometry"]["coordinates"]
        longitude = coordinates[0]
        latitude = coordinates[1]

        rows.append({"Nome": nome, "Longitude": longitude, "Latitude": latitude, "location": Point(longitude, latitude)})

    df = pd.DataFrame(rows)

    gdf = gpd.GeoDataFrame(df, crs=CRS, geometry="location")

    return gdf

In [ ]:
def get_point_of_interest_squares(squares_gdf: gpd.GeoDataFrame, poi_gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """Returns the squares with at least one point of interest.

    Args:
        squares_gdf (gpd.GeoDataFrame): The grid squares GeoDataFrame.
        poi_gdf (gpd.GeoDataFrame): The GeoDataFrame containing the location of each point of interest.

    Returns:
        gpd.GeoDataFrame: The grid squares with at least one point of interest.
    """
    within = gpd.sjoin(poi_gdf, squares_gdf, predicate="within")
    within_counts = within.groupby("index_right").size()
    within_counts.name = "num_poi"
    within_counts = within_counts.reset_index()
    within_counts.set_index("index_right", inplace=True)

    squares_poi_gdf = squares_gdf.merge(within_counts, left_index=True, right_index=True, how="left")
    squares_poi_gdf["num_poi"] = squares_poi_gdf["num_poi"].fillna(0)

    has_poi = squares_poi_gdf["num_poi"] > 0
    
    return squares_gdf[has_poi]

In [ ]:
def calc_distance_to_nearest_square_with_poi(grid_gdf: gpd.GeoDataFrame, grid_poi_gdf: gpd.GeoDataFrame, new_col_name: str) -> gpd.GeoDataFrame:
    """Calculates, for each grid square, the distance (in squares) to the nearest square with a point of interest.

    Args:
        grid_gdf (gpd.GeoDataFrame): The GeoDataFrame with the grid squares center points longitude and latitude.
        grid_poi_gdf (gpd.GeoDataFrame): The GeoDataFrame with the grid squares containing points of interest and their longitude and latitude.
        new_col_name (str): The new column name.

    Returns:
        gpd.GeoDataFrame: Modified grid square's GeoDataFrame with a new column representing the distance to the nearest square with
        a point of interest.
    """
    rad_grid_gdf = grid_gdf.copy()
    rad_points_gdf = grid_poi_gdf.copy()
    
    rad_grid_gdf["longitude"] = np.radians(rad_grid_gdf["longitude"])
    rad_grid_gdf["latitude"] = np.radians(rad_grid_gdf["latitude"])

    rad_points_gdf["longitude"] = np.radians(rad_points_gdf["longitude"])
    rad_points_gdf["latitude"] = np.radians(rad_points_gdf["latitude"])

    dist = DistanceMetric.get_metric('haversine')

    distances = dist.pairwise(rad_grid_gdf[['latitude','longitude']].to_numpy(), rad_points_gdf[['latitude','longitude']].to_numpy()) * 6373
    distances = np.ceil(distances / 0.2)

    distances = np.min(distances, axis=1)

    new_grid_df = grid_gdf.copy()
    new_grid_df[new_col_name] = distances

    return new_grid_df

In [ ]:
# Prepare Metro Dataset
squares_gdf = parse_squares_dataset(grid_path)
metro_gdf = parse_metro_stations_file(metro_station_path)
metro_squares_gdf = get_point_of_interest_squares(squares_gdf, metro_gdf)
metro_dist_gdf = calc_distance_to_nearest_square_with_poi(squares_gdf, metro_squares_gdf, "dist_min_metro")

metro_dist_df = pd.DataFrame(metro_dist_gdf[["grelha_id", "dist_min_metro"]])
metro_dist_df.rename({"grelhad_id": "Grid_ID"}, inplace=True)

In [ ]:
# Prepare Training Dataset
complete_df = mobile_devices.join(metro_dist_df, on="Grid_ID")

complete_df = complete_df[[
        "n_terminals",
        "n_terminals_roaming",
        "terminals_remain",
        "terminals_remain_roaming",
        "in_flow",
        "out_flow",
        "in_flow_roaming",
        "out_flow_roaming",
        "total_flow", 
        'weekday', 
        'time_of_day',
        'dist_min_metro'
    ]]

label_column = 'total_flow'

In [ ]:
# Create Data Split
x_columns = list(complete_df.columns)
x_columns.remove(label_column)

X_train, X_test, y_train, y_test = train_test_split(complete_df[x_columns],complete_df[label_column], train_size=0.8)

train_data = pd.DataFrame(X_train, columns=x_columns)
train_data[label_column] = y_train

test_data = pd.DataFrame(X_test, columns=x_columns)
test_data[label_column] = y_test

In [ ]:
# x_columns = list(mobile_devices.columns)
# x_columns.remove("time_of_day")

# print(x_columns)

# x_train, _, y_train, _ =  train_test_split(
#     list(mobile_devices[x_columns].values),
#     list(mobile_devices["time_of_day"].values),
#     train_size=0.03,
#     stratify=list(mobile_devices["time_of_day"].values)
# )

# mobile_devices_sampled = pd.DataFrame(x_train, columns=x_columns)
# mobile_devices_sampled["time_of_day"] = y_train

# mobile_devices_sampled = mobile_devices.head(1000)

### MODELING

In [ ]:
train_data = lgb.Dataset(train_data, label=label_column)
test_data = lgb.Dataset(test_data, label=label_column)

# Define hyperparameter search space
space = [
    Real(0.001, 0.9, name='learning_rate'),
    Integer(0, 100, name='max_depth'),
    Integer(2, 100, name='num_leaves'),
    Integer(10, 1000, name="n_estimators")
]

expected_flow_mae = complete_df[label_column].mean()
expected_flow_mae


In [ ]:
# Define objective function
def objective(params: typing.Any) -> float:
    """Optimization fuction for the hyper-parameter search.

    Args:
        params (typing.Any): The hyper-parameter values.

    Returns:
        float: The MAE.
    """
    # Extract hyperparameters from params
    learning_rate, max_depth, num_leaves, n_estimators = params

    # Train a LightGBM model with the given hyperparameters
    model = lgb.LGBMRegressor(
        learning_rate=learning_rate,
        max_depth=max_depth,
        num_leaves=num_leaves,
        n_estimators=n_estimators,
        random_state=42
    )


    model.fit(X_train, y_train)

    # Evaluate the model on the test set
    y_pred = model.predict(X_test)
    mse = mean_absolute_error(y_test, y_pred)
    return mse
    
# Run hyperparameter search
result = gp_minimize(objective, space, n_calls=10, random_state=42)
# Print best hyperparameters and corresponding MSE
print(f"Best hyperparameters: {result.x}")
print(f"Corresponding MAE: {result.fun}")

## 🖼️ Visualisations
Copy here the most important visualizations (graphs, charts, maps, images, etc). You can refer to them in the Executive Summary.

Technical note: If not all the visualisations are visible, you can still include them as an image or link - in this case please upload them to your own repository.

In [ ]:
data = pd.read_excel(grid_path, index_col=0)

In [ ]:
DISPLAY_MARKERS = False
DISPLAY_CIRCLES = True
DISPLAY_SQUARES = False
DISPLAY_ONLY_100 = False

m = folium.Map(location=[38.7223, -9.1393], zoom_start="13", tiles="cartodb positron")

for i, item in enumerate(data.values):
    coord = (item[1], item[2])
    c1, c2, backAzimuth = pyproj.Geod(ellps='WGS84').fwd(lons=coord[0],lats= coord[1], az=45, dist=math.sqrt(2*(100**2))) 
    c2_1, c2_2, backAzimuth = pyproj.Geod(ellps='WGS84').fwd(lons=coord[0],lats= coord[1], az=225, dist=math.sqrt(2*(100**2))) 

    # folium.Rectangle([coord, (c1, c2)]).add_to(m)
    # folium.Rectangle([coord, (c1_1, c1_2)]).add_to(m)
    # folium.Rectangle([coord, (c2_1, c2_2)]).add_to(m)
    if DISPLAY_SQUARES: 
        folium.Rectangle([(c2_1, c2_2), (c1, c2)]).add_to(m)

    if DISPLAY_CIRCLES:
        folium.Circle(location=coord,radius=100, fill_color='red').add_to(m)
    if DISPLAY_MARKERS:
        folium.Marker(
            location=[item[1], item[2]]
        ).add_to(m)
    if DISPLAY_ONLY_100:
        if i >= 100:
            break

m

## 👓 References
List all of the external links (even if they are already linked above), such as external datasets, papers, blog posts, code repositories and any other materials.

* [Dataset estações de metro](https://dados.cm-lisboa.pt/dataset/estacoes-de-metro)

## ⏭️ Appendix
Add here any code, images or text that you still find relevant, but that was too long to include in the main report. This section is optional.
